#### Source data is output from BlueCloudAnalytics.  
In BlueCloudAnalytics,
  - Base Reports
    - Checkouts By Date Range (SYM)
      - specify the starting date and ending date
      - save this output file as "Revised Checkouts By Date Range (SYM).csv"


In [ ]:
import requests
import json
import getpass
import csv
from collections import namedtuple
from urllib.parse import quote

### Parse CSV

In [ ]:
def list_to_csv(output_filename, source_list):
    with open(output_filename, 'w') as f:
        for item in source_list:
            headers = item.keys()
            break
        w = csv.DictWriter(f, headers)
        w.writeheader()
        for item_dict in source_list:
            w.writerow(item_dict)
            
def parse_csv(filename):
    item_list = list()
    with open(filename, 'r', newline='', encoding='utf-8') as csvfile:
        csvreader = csv.DictReader(csvfile)
        for item in csvreader:
            item_list.append(item)
    return item_list

In [ ]:
def remove_nonMiddletons(item_list):
    new_list = list()
    for row in item_list:
        if row['Item Library Desc'] == 'Middleton Library (Main Collection)':
            new_list.append(row)
    return new_list

In [ ]:
item_list = parse_csv('Revised Checkouts By Date Range (SYM).csv')
item_list = remove_nonMiddletons(item_list)

####  We enhance the BlueCloud output by adding a call number to each row.  The Symphony API can give us this info.  Here we hit the API with each item.  The response gets merged into the data for each item.

In [ ]:
def get_token():
    baseWsURL = 'https://lalu.sirsi.net/lalu_ilsws/rest'
    clientID = 'DS_CLIENT'
    user = input('what is your Sirsi Workflow username? ')
    password = getpass.getpass('what is your Sirsi Workflow password? ')
    
    login_endpoint = f"{baseWsURL}/security/loginUser?clientID={clientID}&login={user}&password={password}&json=True"

    response = requests.post(login_endpoint)
    response_json = json.loads(response.content)
    return response_json['sessionToken']


def check_connection(token):
    url = f"https://lalu.sirsi.net/lalu_ilsws/rest/security/lookupSessionInfo?clientID=DS_CLIENT&sessionToken={token}&json=True"
    response = requests.post(url)
    print(response.text)


def make_request(service, command, arguments):
    url = f'https://lalu.sirsi.net/lalu_ilsws/rest/{service}/{command}'
    response = requests.post(url, params=arguments)
    return response.text


def lookupItemStatus(itemID, token):
    service = 'circulation'
    command = 'lookupItemStatus'
    arguments = {
        'itemID': itemID,
        'clientID': 'DS_CLIENT',
        'sessionToken': token,
        'json': True,
    }
    return make_request(service, command, arguments)


token = get_token()
check_connection(token)

In [ ]:
for item in item_list:
    api_response = lookupItemStatus(item['Item Barcode'], token)
    api_dict = json.loads(api_response)
    item.update(api_dict)

In [ ]:
for item in item_list:
    item['itemCheckoutInfo'] = ""
    item['itemHoldInfo'] = ""
    item['onreserveInfo'] = ''

In [ ]:
list_to_csv("/home/raleigh/Desktop/FebRanges/plus_api.csv", item_list)

#### Manually remove the iPods, umbrellas, non-Middleton & other items we do not want to count.  Save this csv with the name "Feb_call_numbers.csv".

In [ ]:
item_list = parse_csv('Feb_call_numbers.csv')

In [ ]:
def count_in_range(start, end, item_list):
    count = 0
    for item in item_list:
        if start <= item['callNumber'] <= end[:2]:
            count += 1
    return count

In [ ]:
ranges = [
    ["051", "941"],
    ['B', 'BX'],
    ['CB', 'CT'],
    ['D', 'DT'],
    ['E', 'F'],
    ['G', 'GV'],
    ['H', 'HX'],
    ['JA', 'JZ'],
    ['K', 'KJV'],
    ['LA', 'LJ'],
    ['N', 'NX'],
    ['P', 'PZ'],
    ['Q', 'QR'],
    ['R', 'RS'],
    ['S', 'SH'],
    ['T', 'TX'],
    ['U', 'UG'],
    ['Z', 'ZA'],
]

for start, end in ranges:
    print(f"{start}\t{end}\t{count_in_range(start, end, item_list)}")